## IMDB

In [6]:
'''
Rank / Movie rank order
Title / The title of the film
Genre / A comma-separated list of genres used to classify the film
Description / Brief one-sentence movie summary
Director / The name of the film's director
Actors / A comma-separated list of the main stars of the film
Year / The year that the film released as an integer.
Runtime (Minutes) / The duration of the film in minutes.
Rating / User rating for the movie 0-10
Votes / Number of votes
Revenue (Millions) / Movie revenue in millions
Metascore / An aggregated average of critic scores. Values are between 0 and 100. Higher scores represent positive reviews.
'''

"\nRank / Movie rank order\nTitle / The title of the film\nGenre / A comma-separated list of genres used to classify the film\nDescription / Brief one-sentence movie summary\nDirector / The name of the film's director\nActors / A comma-separated list of the main stars of the film\nYear / The year that the film released as an integer.\nRuntime (Minutes) / The duration of the film in minutes.\nRating / User rating for the movie 0-10\nVotes / Number of votes\nRevenue (Millions) / Movie revenue in millions\nMetascore / An aggregated average of critic scores. Values are between 0 and 100. Higher scores represent positive reviews.\n"

In [3]:
import pandas as pd
metadata = pd.read_csv('IMDB-Movie-Data.csv')
metadata.head(5)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [4]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
Rank                  1000 non-null int64
Title                 1000 non-null object
Genre                 1000 non-null object
Description           1000 non-null object
Director              1000 non-null object
Actors                1000 non-null object
Year                  1000 non-null int64
Runtime (Minutes)     1000 non-null int64
Rating                1000 non-null float64
Votes                 1000 non-null int64
Revenue (Millions)    872 non-null float64
Metascore             936 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 93.8+ KB


### 영화 평점 점수 기준의 수정

인지도가 반영이 되어있지 않음. m을 결정하기 위해 90% percentile 을 사용한다.

In [7]:
C = metadata['Rating'].mean()
C

6.723199999999999

In [9]:
m = metadata['Votes'].quantile(0.9)
m

406251.10000000003

In [11]:
q_movies = metadata.copy().loc[metadata['Votes'] >= m]
q_movies.shape

(100, 12)

### Weight 를 만드는 함수 작성

In [12]:
def WR(x, m = m, C = C) :
    v = x['Votes']
    R = x['Rating']
    return (v/(v+m) * R) + (m/(m+v) * C) # weight 수식을 그대로 대입시킴

In [16]:
metadata['score'] = metadata.apply(WR, axis=1)
metadata.head(5)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,score
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,7.619200
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,6.873945
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,6.884424
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,6.785043
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,6.465695


### 정렬 및 결과확인

In [17]:
col = ['Title', 'Rating', 'Votes', 'score']
rs_original = metadata.sort_values('Rating', ascending = False)[col]
rs_new = metadata.sort_values('score', ascending = False)[col]

In [18]:
rs_original.head(5)

,Title,Rating,Votes,score
54,The Dark Knight,9.0,1791916,8.579216
80,Inception,8.8,1583625,8.376003
117,Dangal,8.8,48969,6.946606
36,Interstellar,8.6,1047747,8.075617
96,Kimi no na wa,8.6,34110,6.868575


In [19]:
rs_new.head(5)

,Title,Rating,Votes,score
54,The Dark Knight,9.0,1791916,8.579216
80,Inception,8.8,1583625,8.376003
36,Interstellar,8.6,1047747,8.075617
124,The Dark Knight Rises,8.5,1222645,8.056861
99,The Departed,8.5,937414,7.962793


### 영화 Description의 유사도를 계산하여 추천

In [20]:
metadata['Description'][0]

'A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.'

In [21]:
metadata['Description'].head()

0    A group of intergalactic criminals are forced ...
1    Following clues to the origin of mankind, a te...
2    Three girls are kidnapped by a man with a diag...
3    In a city of humanoid animals, a hustling thea...
4    A secret government agency recruits some of th...
Name: Description, dtype: object

### Description의 TF-IDF 계산

텍스트의 고유정도를 나타내는 가중치
- null값이 존재함에 따라 fillna 적용

In [25]:
tfIdf = TfidfVectorizer(stop_words = 'english')
metadata.Description = metadata.Description.fillna('')
tfIdf_matrix = tfIdf.fit_transform(metadata.Description)
tfIdf_matrix.shape

(1000, 5667)

In [37]:
print(tfIdf_matrix)

  (0, 2246)	0.21774339622092687
  (0, 2649)	0.33971922781539177
  (0, 1229)	0.33971922781539177
  (0, 2020)	0.24877251686207938
  (0, 5613)	0.2623257870529036
  (0, 4777)	0.2551245011753801
  (0, 1888)	0.3891343606726167
  (0, 5493)	0.2971980892080351
  (0, 4937)	0.3023381228609386
  (0, 1148)	0.3023381228609386
  (0, 5314)	0.3215734366412338
  (1, 2012)	0.29449256448245537
  (1, 1005)	0.30745060337197083
  (1, 3568)	0.37204439281792756
  (1, 3120)	0.3153987761756993
  (1, 4971)	0.22022599122103517
  (1, 1968)	0.20405347429299328
  (1, 4812)	0.37204439281792756
  (1, 1503)	0.3247994693213087
  (1, 3327)	0.3247994693213087
  (1, 4660)	0.24611438520757062
  (1, 4069)	0.2841457702551132
  (2, 2173)	0.24286460105433
  (2, 2825)	0.23873567344604288
  (2, 3108)	0.14914739073502162
  :	:
  (997, 4292)	0.30142341499115827
  (997, 4821)	0.28339339973665945
  (997, 1458)	0.2946733668397855
  (997, 1284)	0.3184321779056564
  (997, 4677)	0.3184321779056564
  (997, 3503)	0.3647509231777795
  (997, 

벡터화된 데이터의 경우, 코사인 유사도가 성능이 가장 좋다.

- TF-IDF의 경우 내적만 구해도 가능함
- sklearn의 cosine_similarities() 대신에 linear_kernel()을 사용한다.

In [26]:
from sklearn.metrics.pairwise import linear_kernel

In [28]:
cosine_sim = linear_kernel(tfIdf_matrix, tfIdf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.04963063],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.04963063, 0.        , ..., 0.        , 0.        ,
        1.        ]])

### 추천 리스트 생성

가장 유사한 10개 영화를 추천리스트로 생성

1. 영화 타이틀을 인덱스화
2. 인덱스를 기준으로 유사도 추출
3. 유사도 기준 10개 영화선정
4. 인덱스를 영화 타이틀로 변환

In [31]:
# 1. 영화 타이틀 index화

indices = pd.Series(metadata.index, index=metadata['Title']).drop_duplicates()
print(indices)

Title
Guardians of the Galaxy                      0
Prometheus                                   1
Split                                        2
Sing                                         3
Suicide Squad                                4
The Great Wall                               5
La La Land                                   6
Mindhorn                                     7
The Lost City of Z                           8
Passengers                                   9
Fantastic Beasts and Where to Find Them     10
Hidden Figures                              11
Rogue One                                   12
Moana                                       13
Colossal                                    14
The Secret Life of Pets                     15
Hacksaw Ridge                               16
Jason Bourne                                17
Lion                                        18
Arrival                                     19
Gold                                        20
Manches

In [34]:
# 4. 인덱스를 영화 타이틀로 변환

def rs(title, n=10, sim=cosine_sim) :
    idx = indices[title]
    sim_score = list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score, key = lambda x: x[1], reverse=True)
    sim_score = sim_score[1 : (n+1)]
    movie_index = [i[0] for i in sim_score]
    return metadata.Title.iloc[movie_index]

In [35]:
rs('Secret in Their Eyes')

181           The VVitch: A New-England Folktale
279                               Iron Man Three
976                                  Dark Places
254                                The Conjuring
687                                       Jumper
800                                  Yoga Hosers
165                                     Twilight
951                              The Descendants
313    The Hobbit: The Battle of the Five Armies
633                             Megan Is Missing
Name: Title, dtype: object

In [36]:
rs('Nine Lives', n=5)

453    Olympus Has Fallen
103          Contratiempo
676           Escape Plan
577                  Tron
781             Self/less
Name: Title, dtype: object

## 내용 기반 추천 시스템 구현

영화 Description 외에 다른 Content 속성을 사용하여 내용 기반 시스템 구현